In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
#gc.collect() で解放



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import random as rnd
# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_log_error

In [ ]:
#from sklearn.metrics import mean_squared_log_errorの使い方
y_true = np.array([1.0, 1.1, 1.3, 1.4, 1.6, 1.7, 1.9, 2.0])
y_pred = np.array([1.0, 1.2, 1.3, 1.5, 1.5, 1.9, 1.9, 2.0])

# scikit-learn で計算する場合
rmsle = np.sqrt(mean_squared_log_error(y_true, y_pred))
print(rmsle)


In [ ]:
train_df = pd.read_table('../input/my-mercari/mercari-price-suggestion-challenge/train.tsv')
test_df = pd.read_table('../input/my-mercari/mercari-price-suggestion-challenge/test.tsv')
test_price=pd.read_csv('../input/sample-submissioncsv/sample_submission.csv')
combine = [train_df,test_df]


In [ ]:
train_df.head()

In [ ]:
train_df.tail()


In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
train_df.item_condition_id.describe()

In [ ]:
train_df.price.describe()
g=sns.FacetGrid(train_df)
g.map(plt.hist, 'price',bins=10)

In [ ]:
new_train=train_df.query('price<300')
g=sns.FacetGrid(new_train)
g.map(plt.hist, 'price',bins=300)

In [ ]:
train_df.item_description[22]

In [ ]:
new_train=train_df.query('price<2')
g=sns.FacetGrid(new_train)
g.map(plt.hist, 'price',bins=300)

In [ ]:
train_df.category_name

In [ ]:
train_df.price.describe()
train_df.plot.scatter(x = 'price',y = 'item_condition_id')

In [ ]:
new_train=train_df.query('price<200')
g=sns.FacetGrid(new_train,col='shipping')
g.map(plt.hist, 'price',bins=30)

In [ ]:
train_copy = train_df.copy()

In [ ]:
train_copy.shape

カテゴリーネームは/で分割されているので分割する。ついでにMainカテゴリー,subカテゴリー,sub2カテゴリーのデータをつくる。

In [ ]:
# Splitting category name
def split_cat(text):
    try: return text.split("/")
    except: return ("No Label", "No Label", "No Label") 
    
# Splitting category name into: Main Category, SubCategory_1, SubCategory_2
train_copy['main_category'], train_copy['subcat_1'], train_copy['subcat_2'] = zip(*train_copy['category_name'].apply(lambda x: split_cat(x)))

In [ ]:
train_copy.head()

In [ ]:
# Label Encoding
from sklearn import preprocessing
def toNumeric(data,to):
    if train_copy[data].dtype == type(object):
        le = preprocessing.LabelEncoder()
        train_copy[to] = le.fit_transform(train_copy[data].astype(str))   
toNumeric('name','n_name')
toNumeric('category_name','n_category_name')
toNumeric('brand_name','n_brand_name')
toNumeric('main_category','n_main_category')
toNumeric('subcat_1','n_subcat_1')
toNumeric('subcat_2','n_subcat_2')
train_copy.head()

In [ ]:
#Checking for NULL values in the columns
train_copy.isnull().any()

category_name,brand_name,item_descriptionに欠損値があるので、欠損値を埋める。

In [ ]:
def fill_missing_data(data):
    data.category_name.fillna(value = "Other/Other/Other", inplace = True)
    data.brand_name.fillna(value = "Unknown brand", inplace = True)
    data.item_description.fillna(value = "No description", inplace = True)
    return data

train_copy = fill_missing_data(train_copy)
train_copy = train_copy.dropna()
print(np.shape(train_copy))
train_copy.head()

In [ ]:
priceに関して表示

In [ ]:
print("Range of price : ")
print("Minimum Price: ",'$', train_copy["price"].min())
print("Maximum Price: ",'$', train_copy["price"].max())
fig, ax = plt.subplots(3, 1, figsize = (13, 16))
ax[0].hist(train_copy.price, bins = 100, range = [min(train_copy.price), max(train_copy.price)+100], label = "price", color='red', alpha=0.7)
ax[0].annotate(' Outliers\n present\n till\n this point', xy=(max(train_copy.price), 100), xytext=(max(train_copy.price), 125000), arrowprops=dict(facecolor='black'), color='black')
ax[0].set_title("Histogram of Price Distribution", fontsize = 13)
ax[0].set_xlabel("Price", fontsize = 10)
ax[0].set_ylabel("Frequency ", fontsize = 10)

ax[1].set_title("Histogram of Price Distribution (Focused Mode)", fontsize = 13)
ax[1].hist(train_copy.price, bins = 100, range = [0, 200], label = "price", color='red', alpha=0.7)
ax[1].set_xlabel("Price", fontsize = 10)
ax[1].set_ylabel("Frequency ", fontsize = 10)


sns.boxplot(train_copy.price, showfliers = False, ax = ax[2], linewidth=0.7, color='red')
ax[2].set_title("Box Plot for Price Distribution", fontsize = 13)
ax[2].set_xlabel("Price", fontsize = 10)
plt.show()

ブランドに関しての表示

In [ ]:
brands = train_copy["brand_name"].value_counts()
print("No. of Unique Brand Names :", brands.size)
fig, ax = plt.subplots(1, 2, figsize = (13, 6))
# we skipped '0' index and started from 1st because 0th index has "unknown brands"
sns.barplot(brands[1:11].values, brands[1:11].index,ax = ax[0], edgecolor='k', linewidth=0.5, palette='rocket') 
ax[0].set_title("Top 10 Most Frequently Used Brand Names", fontsize = 13)
ax[0].set_xlabel("Counts", fontsize = 10)
ax[0].set_ylabel("Brand Name", fontsize = 10)

import pandas as pd
top10_brands = train_copy.groupby('brand_name', axis=0).mean()
df_expPrice = pd.DataFrame(top10_brands.sort_values('price', ascending = False)['price'][0:10].reset_index())
ax[1].set_title("Top 10 Most Costly Brands", fontsize = 13)
ax[1] = sns.barplot(x="brand_name", y="price", data=df_expPrice, edgecolor='k', linewidth=0.5, palette='PuRd')
ax[1].set_xlabel("Brand Name", fontsize = 10)
ax[1].set_ylabel("Sale Price", fontsize = 10)
ax[1].set_xticklabels(ax[1].get_xticklabels(),rotation=35)
plt.show()

アイテムコンディションに関する表示

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (13,6))
sns.countplot(train_copy.item_condition_id, ax = ax[0], palette='Blues_r', edgecolor='k', linewidth=0.6)
rectangles = ax[0].patches
ax[0].set_title("Count Distribution of Item Condition Ids", fontsize = 13)
labels = train_copy.item_condition_id.value_counts().values
for rect, label in zip(rectangles, labels):
    height = rect.get_height()
    ax[0].text(rect.get_x() + rect.get_width()/2, height + 5, label, ha = "center", va = "bottom")
ax[1].set_title("Sale Price Distribution of Item Condition Ids", fontsize = 13)    
sns.boxplot(x = train_copy.item_condition_id, y = train_copy.price, showfliers = False, orient = "v", ax = ax[1], hue = train_copy.shipping, palette="Set1", linewidth=0.6)
plt.show()

メインカテゴリーに関する表示

In [ ]:
fig, ax = plt.subplots(2, 1, figsize = (13,18))
sns.countplot(train_copy.main_category, ax = ax[0], palette='Reds', edgecolor='k', linewidth=0.6)
ax[0].set_xticklabels(ax[0].get_xticklabels(),rotation=15)
ax[0].set_title("Count Distribution of Main Categories", fontsize = 13)
rectangles = ax[0].patches
labels = train_copy.main_category.value_counts().values
for rect, label in zip(rectangles, labels):
    height = rect.get_height()
    ax[0].text(rect.get_x() + rect.get_width()/2, height + 5, label, ha = "center", va = "bottom")
sns.boxplot(x = train_copy.main_category, y = train_copy.price, showfliers = False, orient = "v", ax = ax[1], hue = train_copy.shipping, palette="Set1", linewidth=0.6)
ax[1].set_xticklabels(ax[1].get_xticklabels(),rotation=15)
ax[1].set_title("Sale Price Distribution of Main Categories", fontsize = 13)
plt.show()

製品紹介に関する表示

In [ ]:
#python -m pip install wordcloud
from wordcloud import WordCloud
import os
wordcloud = WordCloud(width = 2400, height = 1200).generate(" ".join(train_copy.item_description.astype(str)))
plt.figure(figsize = (13, 10))
plt.imshow(wordcloud)
plt.show()

In [ ]:
rand_train=train_df.sample(frac=0.1)

In [ ]:
rand_train.head

In [ ]:
rand_train.describe()

In [ ]:
rand_train.price

In [ ]:
rand_train.head

In [ ]:
rand_train.category_name

In [ ]:
rand_train['brand_name'].value_counts

In [ ]:
train_df.isna().sum()


In [ ]:
rand_train.isna().sum()

In [ ]:
train_Ter = rand_train.drop(['name','category_name','brand_name','item_description'],axis=1)

In [ ]:
train_Ter.head

In [ ]:
X_train = train_Ter.drop("price",axis=1)
Y_train = train_Ter['price'].astype(int)
X_test = Y_train
Y_train,Y_train,X_test

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train,Y_train)
Y_pred = logreg.predict(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_train)


In [ ]:
Y_pred.shape,X_test.shape

In [ ]:
rmsle = np.sqrt(mean_squared_log_error(X_test,Y_pred))
print(rmsle)